# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_excertos.CSV', sep = ';')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_EXCERTO,TEXTO_EXCERTO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995438,Voto:Cuidam os autos de tomada de contas espec...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025603,Voto:Cuidam os autos de Solicitação do Congres...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455375,Relatório:Trata-se de embargos de declaração o...,Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773747,Voto:8. Em relação a outros processos judiciai...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773403,Voto:11. Relativamente ao ato envolvendo a Sra...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13285, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          553
Contrato Administrativo     941
Convênio                    683
Desestatização              139
Direito Processual         1811
Finanças Públicas           328
Gestão Administrativa       338
Licitação                  2756
Pessoal                    3393
Responsabilidade           2343
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13285, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 2000
dim_vetor = 50

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_EXCERTO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_EXCERTO'])

Using TensorFlow backend.


Found 63925 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13285, 2000)


In [9]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('../externos/model-50.txt')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 63925
Encontrados no modelo: 42309 = 66.18537348455222


# Treinamento

## Embedding com pesos fixos

In [11]:
from keras.layers import Embedding

embedding = Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False)

W1106 11:20:43.069194 140038781339456 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers.core import Dropout

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

W1106 11:20:48.618535 140038781339456 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1106 11:20:48.708407 140038781339456 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1106 11:20:48.737226 140038781339456 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1106 11:20:48.739520 140038781339456 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
flatten_1 (Flatten)          (None, 100000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               51200512  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total para

In [13]:
history = model.fit(x, y, epochs=50, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1106 11:21:05.274867 140038781339456 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10628 samples, validate on 2657 samples
Epoch 1/50
10628/10628 [==============================] - 23s 2ms/step - loss: 2.5416 - categorical_accuracy: 0.2513 - val_loss: 2.1102 - val_categorical_accuracy: 0.1701
Epoch 2/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.9641 - categorical_accuracy: 0.2867 - val_loss: 2.0312 - val_categorical_accuracy: 0.1682
Epoch 3/50
10628/10628 [==============================] - 21s 2ms/step - loss: 1.8802 - categorical_accuracy: 0.3433 - val_loss: 2.0073 - val_categorical_accuracy: 0.1675
Epoch 4/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.8101 - categorical_accuracy: 0.3787 - val_loss: 1.8847 - val_categorical_accuracy: 0.2699
Epoch 5/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.7697 - categorical_accuracy: 0.4010 - val_loss: 1.8717 - val_categorical_accuracy: 0.2458
Epoch 6/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.7154 - categori

10628/10628 [==============================] - 20s 2ms/step - loss: 0.9633 - categorical_accuracy: 0.6583 - val_loss: 1.9976 - val_categorical_accuracy: 0.4964
Epoch 49/50
10628/10628 [==============================] - 20s 2ms/step - loss: 0.9587 - categorical_accuracy: 0.6636 - val_loss: 2.0658 - val_categorical_accuracy: 0.4784
Epoch 50/50
10628/10628 [==============================] - 20s 2ms/step - loss: 0.9476 - categorical_accuracy: 0.6644 - val_loss: 2.0741 - val_categorical_accuracy: 0.4377


In [14]:
model = None
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=50, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/50
10628/10628 [==============================] - 22s 2ms/step - loss: 2.3685 - categorical_accuracy: 0.2837 - val_loss: 2.0772 - val_categorical_accuracy: 0.1577
Epoch 2/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.7336 - categorical_accuracy: 0.4182 - val_loss: 1.9333 - val_categorical_accuracy: 0.2510
Epoch 3/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.5173 - categorical_accuracy: 0.4915 - val_loss: 1.7360 - val_categorical_accuracy: 0.3244
Epoch 4/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.3745 - categorical_accuracy: 0.5372 - val_loss: 1.7770 - val_categorical_accuracy: 0.3293
Epoch 5/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.2341 - categorical_accuracy: 0.5793 - val_loss: 1.9187 - val_categorical_accuracy: 0.3515
Epoch 6/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.1277 - categori

10628/10628 [==============================] - 20s 2ms/step - loss: 0.2914 - categorical_accuracy: 0.9059 - val_loss: 2.1304 - val_categorical_accuracy: 0.4136
Epoch 49/50
10628/10628 [==============================] - 20s 2ms/step - loss: 0.2699 - categorical_accuracy: 0.9094 - val_loss: 2.2890 - val_categorical_accuracy: 0.4279
Epoch 50/50
10628/10628 [==============================] - 20s 2ms/step - loss: 0.2809 - categorical_accuracy: 0.9095 - val_loss: 3.1508 - val_categorical_accuracy: 0.4249


In [15]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=50, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/50
10628/10628 [==============================] - 21s 2ms/step - loss: 2.3149 - categorical_accuracy: 0.2906 - val_loss: 2.0455 - val_categorical_accuracy: 0.2315
Epoch 2/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.6704 - categorical_accuracy: 0.4419 - val_loss: 2.1492 - val_categorical_accuracy: 0.2499
Epoch 3/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.4121 - categorical_accuracy: 0.5230 - val_loss: 1.8090 - val_categorical_accuracy: 0.3703
Epoch 4/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.2363 - categorical_accuracy: 0.5725 - val_loss: 1.6801 - val_categorical_accuracy: 0.4084
Epoch 5/50
10628/10628 [==============================] - 20s 2ms/step - loss: 1.1080 - categorical_accuracy: 0.6166 - val_loss: 1.6931 - val_categorical_accuracy: 0.4482
Epoch 6/50
10628/10628 [==============================] - 20s 2ms/step - loss: 0.9835 - categori

10628/10628 [==============================] - 20s 2ms/step - loss: 0.2805 - categorical_accuracy: 0.9089 - val_loss: 2.9312 - val_categorical_accuracy: 0.4633
Epoch 49/50
10628/10628 [==============================] - 20s 2ms/step - loss: 0.2651 - categorical_accuracy: 0.9135 - val_loss: 2.6672 - val_categorical_accuracy: 0.4543
Epoch 50/50
10628/10628 [==============================] - 20s 2ms/step - loss: 0.2561 - categorical_accuracy: 0.9186 - val_loss: 2.8634 - val_categorical_accuracy: 0.4324


## Embedding com pesos ajustáveis

In [16]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=50, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1106 12:11:13.113506 140038781339456 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1106 12:11:13.138351 140038781339456 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
flatten_4 (Flatten)          (None, 100000)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               51200512  
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                2570      
Total para

10628/10628 [==============================] - 22s 2ms/step - loss: 0.8785 - categorical_accuracy: 0.7109 - val_loss: 1.9352 - val_categorical_accuracy: 0.5382
Epoch 42/50
10628/10628 [==============================] - 22s 2ms/step - loss: 0.9063 - categorical_accuracy: 0.7085 - val_loss: 2.0358 - val_categorical_accuracy: 0.5502
Epoch 43/50
10628/10628 [==============================] - 22s 2ms/step - loss: 0.8891 - categorical_accuracy: 0.7068 - val_loss: 2.0473 - val_categorical_accuracy: 0.5491
Epoch 44/50
10628/10628 [==============================] - 22s 2ms/step - loss: 0.9169 - categorical_accuracy: 0.7062 - val_loss: 1.9674 - val_categorical_accuracy: 0.5578
Epoch 45/50
10628/10628 [==============================] - 22s 2ms/step - loss: 0.8940 - categorical_accuracy: 0.7021 - val_loss: 1.8722 - val_categorical_accuracy: 0.5367
Epoch 46/50
10628/10628 [==============================] - 22s 2ms/step - loss: 0.8459 - categorical_accuracy: 0.7158 - val_loss: 2.1002 - val_categoric

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=50, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/50
10628/10628 [==============================] - 23s 2ms/step - loss: 2.2494 - categorical_accuracy: 0.3079 - val_loss: 1.9634 - val_categorical_accuracy: 0.2480
Epoch 2/50
10628/10628 [==============================] - 22s 2ms/step - loss: 1.6013 - categorical_accuracy: 0.4655 - val_loss: 1.6643 - val_categorical_accuracy: 0.3357
Epoch 3/50
10628/10628 [==============================] - 22s 2ms/step - loss: 1.3240 - categorical_accuracy: 0.5598 - val_loss: 1.6458 - val_categorical_accuracy: 0.4370
Epoch 4/50
10628/10628 [==============================] - 22s 2ms/step - loss: 1.1221 - categorical_accuracy: 0.6209 - val_loss: 1.5903 - val_categorical_accuracy: 0.5092
Epoch 5/50
10628/10628 [==============================] - 22s 2ms/step - loss: 0.9848 - categorical_accuracy: 0.6602 - val_loss: 1.4234 - val_categorical_accuracy: 0.5634
Epoch 6/50
10628/10628 [==============================] - 22s 2ms/step - loss: 0.8686 - categori

10628/10628 [==============================] - 22s 2ms/step - loss: 0.1830 - categorical_accuracy: 0.9499 - val_loss: 2.5114 - val_categorical_accuracy: 0.5243
Epoch 49/50
10628/10628 [==============================] - 22s 2ms/step - loss: 0.1727 - categorical_accuracy: 0.9499 - val_loss: 3.1302 - val_categorical_accuracy: 0.5536
Epoch 50/50
10628/10628 [==============================] - 22s 2ms/step - loss: 0.1790 - categorical_accuracy: 0.9503 - val_loss: 2.3291 - val_categorical_accuracy: 0.5593


In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=50, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/50
10628/10628 [==============================] - 13s 1ms/step - loss: 2.1408 - categorical_accuracy: 0.2833 - val_loss: 1.9593 - val_categorical_accuracy: 0.2296
Epoch 2/50
10628/10628 [==============================] - 12s 1ms/step - loss: 1.7429 - categorical_accuracy: 0.3963 - val_loss: 2.6276 - val_categorical_accuracy: 0.2055
Epoch 3/50
10628/10628 [==============================] - 12s 1ms/step - loss: 1.5245 - categorical_accuracy: 0.4837 - val_loss: 2.3128 - val_categorical_accuracy: 0.2251
Epoch 4/50
10628/10628 [==============================] - 12s 1ms/step - loss: 1.3617 - categorical_accuracy: 0.5327 - val_loss: 1.5825 - val_categorical_accuracy: 0.4264
Epoch 5/50
10628/10628 [==============================] - 12s 1ms/step - loss: 1.2265 - categorical_accuracy: 0.5734 - val_loss: 1.5616 - val_categorical_accuracy: 0.4550
Epoch 6/50
10628/10628 [==============================] - 12s 1ms/step - loss: 1.1246 - categori

10628/10628 [==============================] - 12s 1ms/step - loss: 0.3758 - categorical_accuracy: 0.8911 - val_loss: 2.1095 - val_categorical_accuracy: 0.5713
Epoch 49/50
10628/10628 [==============================] - 12s 1ms/step - loss: 0.3658 - categorical_accuracy: 0.8870 - val_loss: 2.3466 - val_categorical_accuracy: 0.5262
Epoch 50/50
10628/10628 [==============================] - 12s 1ms/step - loss: 0.3678 - categorical_accuracy: 0.8894 - val_loss: 2.2482 - val_categorical_accuracy: 0.5766


In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[100000,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node training_6/Adadelta/Variable_1/Assign (defined at /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:402) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Errors may have originated from an input operation.
Input Source operations connected to node training_6/Adadelta/Variable_1/Assign:
 training_6/Adadelta/zeros_1 (defined at /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:702)

Original stack trace for 'training_6/Adadelta/Variable_1/Assign':
  File "/home/leonardo/anaconda3/envs/gpu/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/leonardo/anaconda3/envs/gpu/lib/pyt

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor of shape [100000,1024] and type float
	 [[node training_7/Adadelta/zeros_8 (defined at /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:702) ]]

Original stack trace for 'training_7/Adadelta/zeros_8':
  File "/home/leonardo/anaconda3/envs/gpu/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/leonardo/anaconda3/envs/gpu/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/leonardo/